# Launch Sagemaker Pipeline

In [2]:
import boto3
import sagemaker
from datetime import date
from sagemaker.workflow.parallelism_config import ParallelismConfiguration
import utils

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


## General parameters

In [3]:
 # By default
region = boto3.Session().region_name
role = sagemaker.get_execution_role()

# Use own bucket and prefix
s3_bucket_nps = 'iberia-data-lake' # In this case: iberia-data-lake
s3_bucket_nps_prefix = 'customer/nps_explainability_model' # In this case: sagemaker/sagemaker-template

s3_bucket_lf = 'ibdata-prod-ew1-s3-customer'
s3_bucket_lf_prefix = 'customer/load_factor_to_s3_nps_model'

# Environment (sbx or prod pipeline)
environment = "develop"
environment_ppl = "prod" if environment == "production" else "sbx"

# Config file
configuration = utils.read_config_data()

# Parameters
str_execution_date = '2024-03-13' # Change
is_last_date = "0" # Optional
model_version = configuration.get("MODEL_VERSION")
vertical = configuration.get("VERTICAL")
s3_path_write = "customer/nps_explainability_model/sbx"


insert_date_ci='2024-02-12'
s3_path_read_nps = 'customer/nps_surveys/export_historic'
s3_path_read_lf = "customer/load_factor_to_s3_nps_model"



sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


## Pipeline definition

### Create pipeline execution updating code from sagemaker repository

In [4]:
from production.pipelines_code.pipeline import get_pipeline as pipeline_definition

pipeline_name = (
    f"ibdata-{vertical}-{model_version.lower()}-{environment_ppl}-ppl"
)
base_job_prefix = pipeline_name

pipe_definition = pipeline_definition(
    region=region,
    role=role,
    default_bucket='iberia-data-lake',
    default_bucket_prefix='customer/nps_explainability_model/sbx/pipeline',
    pipeline_name=pipeline_name,
    base_job_prefix=base_job_prefix
)
pipe_definition.upsert(role_arn=role)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


INFO:sagemaker.processing:Uploaded None to s3://iberia-data-lake/customer/nps_explainability_model/sbx/pipeline/ibdata-customer-nps-client-model-sbx-pp-2024-03-25-16-28-39-036/source/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://iberia-data-lake/customer/nps_explainability_model/sbx/pipeline/ibdata-customer-nps-client-model-sbx-pp-2024-03-25-16-28-39-036/source/runproc.sh


Using provided s3_resource


INFO:sagemaker.processing:Uploaded None to s3://iberia-data-lake/customer/nps_explainability_model/sbx/pipeline/ibdata-customer-nps-client-model-sbx-pp-2024-03-25-16-28-39-552/source/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://iberia-data-lake/customer/nps_explainability_model/sbx/pipeline/ibdata-customer-nps-client-model-sbx-pp-2024-03-25-16-28-39-552/source/runproc.sh


Using provided s3_resource


INFO:sagemaker.processing:Uploaded None to s3://iberia-data-lake/customer/nps_explainability_model/sbx/pipeline/ibdata-customer-nps-client-model-sbx-pp-2024-03-25-16-28-39-738/source/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://iberia-data-lake/customer/nps_explainability_model/sbx/pipeline/ibdata-customer-nps-client-model-sbx-pp-2024-03-25-16-28-39-738/source/runproc.sh
INFO:sagemaker.processing:Uploaded None to s3://iberia-data-lake/customer/nps_explainability_model/sbx/pipeline/ibdata-customer-nps-client-model-sbx-pp-2024-03-25-16-28-39-894/source/sourcedir.tar.gz


Using provided s3_resource
Using provided s3_resource


INFO:sagemaker.processing:runproc.sh uploaded to s3://iberia-data-lake/customer/nps_explainability_model/sbx/pipeline/ibdata-customer-nps-client-model-sbx-pp-2024-03-25-16-28-39-894/source/runproc.sh
INFO:sagemaker.processing:Uploaded None to s3://iberia-data-lake/customer/nps_explainability_model/sbx/pipeline/ibdata-customer-nps-client-model-sbx-pp-2024-03-25-16-28-40-064/source/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://iberia-data-lake/customer/nps_explainability_model/sbx/pipeline/ibdata-customer-nps-client-model-sbx-pp-2024-03-25-16-28-40-064/source/runproc.sh


Using provided s3_resource


{'PipelineArn': 'arn:aws:sagemaker:eu-west-1:077156906314:pipeline/ibdata-customer-nps-client-model-sbx-ppl',
 'ResponseMetadata': {'RequestId': 'b316f9d1-d50f-46f2-ab9f-9e09c118d287',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b316f9d1-d50f-46f2-ab9f-9e09c118d287',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '108',
   'date': 'Mon, 25 Mar 2024 16:28:40 GMT'},
  'RetryAttempts': 0}}

In [5]:
pipeline_name

'ibdata-customer-nps-client-model-sbx-ppl'

### Execute pipeline with parameters

In [6]:
# This cell starts an execution of the previous pipeline created
execution = pipe_definition.start(
    parallelism_config=ParallelismConfiguration(max_parallel_execution_steps=4).to_request(),
    parameters=dict(
        str_execution_date=str_execution_date,
        is_last_date=is_last_date,
        s3_path_write=s3_path_write,
        s3_path_read_nps=s3_path_read_nps,
        s3_path_read_lf=s3_path_read_lf,        
        use_type='train'
    )
)
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:eu-west-1:077156906314:pipeline/ibdata-customer-nps-client-model-sbx-ppl',
 'PipelineExecutionArn': 'arn:aws:sagemaker:eu-west-1:077156906314:pipeline/ibdata-customer-nps-client-model-sbx-ppl/execution/sr4l1spwpl6q',
 'PipelineExecutionDisplayName': 'execution-1711384121417',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2024, 3, 25, 16, 28, 41, 322000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 3, 25, 16, 28, 41, 322000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:eu-west-1:077156906314:user-profile/d-8rnxtw3g36nb/customer-diego-rivera',
  'UserProfileName': 'customer-diego-rivera',
  'DomainId': 'd-8rnxtw3g36nb',
  'IamIdentity': {'Arn': 'arn:aws:sts::077156906314:assumed-role/ibdata-aip-role-sagemaker-customer-user/SageMaker',
   'PrincipalId': 'AROARD5XI6VFDJH3ORBAM:SageMaker'}},
 'LastModifiedBy': {'UserProfileArn': 'arn:aws:sagemaker:eu-west-1:077156906314:user-profile

In [7]:
import pandas as pd